# **CIFAR10 AND CIFAR100 CLASSIFICATION USING ALEXNET WITH CUTOUT**

### **Importing the relevant libraries**

In [1]:
# Imports and Setup
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, SubsetRandomSampler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### **CUTOUT IMPLEMENATION**

In [3]:
# Define the cutout function
def cutout(image, n_holes=1, length=16):
    h, w = image.size(1), image.size(2)
    mask = np.ones((h, w), np.float32)

    for _ in range(n_holes):
        y = np.random.randint(h)
        x = np.random.randint(w)

        y1 = np.clip(y - length // 2, 0, h)
        y2 = np.clip(y + length // 2, 0, h)
        x1 = np.clip(x - length // 2, 0, w)
        x2 = np.clip(x + length // 2, 0, w)

        mask[y1: y2, x1: x2] = 0.

    mask = torch.from_numpy(mask).expand_as(image)
    image = image * mask

    return image

# Define a lambda function to apply cutout
apply_cutout = lambda img: cutout(img, n_holes=1, length=16)

### **DATA LOADING AND TRANSFORMATION**

In [4]:
# Image augmentation and transformation for training
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Lambda(apply_cutout) # We applied the cut out function here.
])

# Transformation for validation and test
valid_test_transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Prepare the dataset for training and validation
data_directory = './data_path_cifar10'

# Load CIFAR-10 dataset once
cifar10_dataset = datasets.CIFAR10(root=data_directory, download=True, transform=None)

# Apply different transforms to train, validation, and test splits
train_data = datasets.CIFAR10(root=data_directory, train=True, download=False, transform=train_transform)
valid_data = datasets.CIFAR10(root=data_directory, train=True, download=False, transform=valid_test_transform)
test_data = datasets.CIFAR10(root=data_directory, train=False, download=False, transform=valid_test_transform)

# Split the training data for validation
valid_split, shuffle, random_seed = 0.1, True, 42
data_size = len(train_data)
indices = list(range(data_size))
split = int(np.floor(valid_split * data_size))

if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, valid_indices = indices[split:], indices[:split]

# Creating the data loader for training and validation
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

print(f'Training samples: {len(train_indices)}')
print(f'Validation samples: {len(valid_indices)}')
print(f'Testing samples: {len(test_data)}')


100%|██████████| 170498071/170498071 [00:04<00:00, 35177585.64it/s]


Extracting ./data_path_cifar10/cifar-10-python.tar.gz to ./data_path_cifar10
Training samples: 45000
Validation samples: 5000
Testing samples: 10000


Checking shapes for features and target

In [5]:
# Get a single batch from the train_loader
data_iter = iter(train_loader)
features, targets = next(data_iter)

# Print the shapes of the features and targets
print(f'Shape of features: {features.shape}')
print(f'Shape of targets: {targets.shape}')


Shape of features: torch.Size([64, 3, 224, 224])
Shape of targets: torch.Size([64])


Implementing AlexNet model from scratch

In [6]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()

        # NB:CIFAR-10 is resized to 224x224
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # N x 96 x 55 x 55
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # N x 96 x 27 x 27

            nn.Conv2d(96, 256, kernel_size=5, padding=2),  # N x 256 x 27 x 27
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # N x 256 x 13 x 13

            nn.Conv2d(256, 384, kernel_size=3, padding=1),  # N x 384 x 13 x 13
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # N x 384 x 13 x 13
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # N x 256 x 13 x 13
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)  # N x 256 x 6 x 6
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)
        )

#         self._initialize_weights()

#     def _initialize_weights(self):
#         for layers in self.modules():
#             if isinstance(layers, nn.Conv2d):
#                 nn.init.normal_(layers.weight, mean=0, std=0.01)
#                 if layers.bias is not None:
#                     nn.init.constant_(layers.bias, 0)

#             elif isinstance(layers, nn.Linear):
#                 nn.init.normal_(layers.weight, mean=0, std=0.01)
#                 if layers.bias is not None:
#                     nn.init.constant_(layers.bias, 1)


    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

Training and testing function

In [7]:
def train(model, criterion, train_loader, valid_loader, optimizer, lr_scheduler, num_epochs):
    model = model.to(device)
    ema_loss = None
    best_acc = 0
    losses = []
    training_accuracies = []
    validation_accuracies = []

    print('Training Loop with Cutout... ')

    for epoch in range(num_epochs):
        model.train()
        correct = 0

        for batch_idx, (features, target) in enumerate(train_loader):
            features, target = features.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(features) # pass input feature for prediction
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                pred = output.argmax(dim=1)
                correct += pred.eq(target).sum().item()

            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01

        train_score = correct / len(train_loader.dataset)
        _, _, valid_score = test(model, valid_loader, mode='Validation')

        losses.append(ema_loss)
        training_accuracies.append(train_score)
        validation_accuracies.append(valid_score)

        if valid_score > best_acc:
            best_acc = valid_score

        print(f'Epoch: {epoch + 1} \tLoss: {ema_loss:.4f} \tTraining Accuracy: {train_score:.4f} \tValidation Accuracy: {valid_score:.4f}')

        lr_scheduler.step()

    return losses, training_accuracies, validation_accuracies


def test(model, data_loader, mode='Test'):
    model.eval()
    correct_top1 = 0
    correct_top5 = 0
    correct = 0
    total = 0

    print(f'------{mode} Loop --------')

    with torch.no_grad():
        for features, target in data_loader:
            features, target = features.to(device), target.to(device)
            outputs = model(features)
            pred = outputs.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

            _, pred_top1 = outputs.topk(1, dim=1, largest=True, sorted=True)
            correct_top1 += (pred_top1.view(-1) == target).sum().item()

            _, pred_top5 = outputs.topk(5, dim=1, largest=True, sorted=True)
            correct_top5 += (pred_top5 == target.view(-1, 1)).sum().item()

    top1_error = 1 - correct_top1 / total
    top5_error = 1 - correct_top5 / total
    accuracy = correct / total

    return top1_error, top5_error, accuracy

Training model

In [8]:
# Assuming train_loader and valid_loader are defined

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 10  # Adjust as needed

model10 = AlexNet(num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()

learning_rate, weight_decay, momentum = 0.01, 0.0005, 0.9
optimizer = torch.optim.SGD(model10.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

# Approach for learning rate decay
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Train the model
num_epochs = 25
losses, training_accuracies, validation_accuracies = train(
    model10, criterion, train_loader, valid_loader, optimizer, lr_scheduler, num_epochs
)

# Save the model weights
torch.save(model10.state_dict(), 'alexnet_10_weights_cutout.pth')
torch.save(model10, 'alexnet_10_model_cutout.pth') # save the entire model
print(f'\nModel weights for CIFAR10 saved successfully!')

Training Loop with Cutout... 
------Validation Loop --------
Epoch: 1 	Loss: 1.8492 	Training Accuracy: 0.1836 	Validation Accuracy: 0.2740
------Validation Loop --------
Epoch: 2 	Loss: 1.4257 	Training Accuracy: 0.3863 	Validation Accuracy: 0.4920
------Validation Loop --------
Epoch: 3 	Loss: 1.1856 	Training Accuracy: 0.4911 	Validation Accuracy: 0.6092
------Validation Loop --------
Epoch: 4 	Loss: 1.0143 	Training Accuracy: 0.5633 	Validation Accuracy: 0.6648
------Validation Loop --------
Epoch: 5 	Loss: 0.8713 	Training Accuracy: 0.6098 	Validation Accuracy: 0.7112
------Validation Loop --------
Epoch: 6 	Loss: 0.8001 	Training Accuracy: 0.6411 	Validation Accuracy: 0.7364
------Validation Loop --------
Epoch: 7 	Loss: 0.7396 	Training Accuracy: 0.6637 	Validation Accuracy: 0.7672
------Validation Loop --------
Epoch: 8 	Loss: 0.6638 	Training Accuracy: 0.6866 	Validation Accuracy: 0.7700
------Validation Loop --------
Epoch: 9 	Loss: 0.6445 	Training Accuracy: 0.6986 	Validati

Plotting accuracies and loss 

In [9]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a subplot with two rows and one column
fig = make_subplots(rows=2, cols=1, subplot_titles=("Accuracy", "Loss"))

# Add training accuracy to the first subplot
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=training_accuracies, mode='lines', name='Train Accuracy'), row=1, col=1)

# Add validation accuracy to the first subplot
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=validation_accuracies, mode='lines', name='Valid Accuracy'), row=1, col=1)

# Add training loss to the second subplot
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=losses, mode='lines', name='Loss'), row=2, col=1)

# Update the layout with the title
fig.update_layout(title='Model Accuracy and Loss - CutOut (CIFAR10)')

# Show the figure
fig.show()
# Save the imnage
plt.savefig('accuracy_plot_CutOut_cifar10.png')

<Figure size 640x480 with 0 Axes>

Evaluating model

In [11]:
# Evaluate the Model
model10 = AlexNet(num_classes=NUM_CLASSES).to(device)
model10.load_state_dict(torch.load('/kaggle/working/alexnet_10_weights_cutout.pth'))
model10.eval()

# Evaluate the model on the test set
top1_error, top5_error, test_accuracy = test(model10, test_loader, mode='Test')
print(f"Top-1 Error: {top1_error * 100:.2f}%,  Top-5 Error: {top5_error * 100:.2f}%,  Test Accuracy: {test_accuracy*100:.2f}%")

------Test Loop --------
Top-1 Error: 13.38%,  Top-5 Error: 0.58%,  Test Accuracy: 86.62%


# **CIFAR100 CLASSIFICATIOIN ALEXNET ARCHITECTURE**

Downloading and transforming data for cifar100

In [12]:
# Image augmentation and transformation for training
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Lambda(apply_cutout) # We applied the cut out function here.
])

# Transformation for validation and test
valid_test_transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Prepare the dataset for training and validation
data_directory = './data_path_cifar100'

# Load CIFAR-10 dataset once
cifar100_dataset = datasets.CIFAR100(root=data_directory, download=True, transform=None)

# Apply different transforms to train, validation, and test splits
train_data = datasets.CIFAR100(root=data_directory, train=True, download=False, transform=train_transform)
valid_data = datasets.CIFAR100(root=data_directory, train=True, download=False, transform=valid_test_transform)
test_data = datasets.CIFAR100(root=data_directory, train=False, download=False, transform=valid_test_transform)

# Split the training data for validation
valid_split, shuffle, random_seed = 0.1, True, 42
data_size = len(train_data)
indices = list(range(data_size))
split = int(np.floor(valid_split * data_size))

if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, valid_indices = indices[split:], indices[:split]

# Creating the data loader for training and validation
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=64, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

print(f'Training samples: {len(train_indices)}')
print(f'Validation samples: {len(valid_indices)}')
print(f'Testing samples: {len(test_data)}')


100%|██████████| 169001437/169001437 [00:03<00:00, 49066529.96it/s]


Extracting ./data_path_cifar100/cifar-100-python.tar.gz to ./data_path_cifar100
Training samples: 45000
Validation samples: 5000
Testing samples: 10000


Alexnet model

In [13]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # N x 96 x 55 x 55
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # N x 96 x 27 x 27

            nn.Conv2d(96, 256, kernel_size=5, padding=2),  # N x 256 x 27 x 27
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # N x 256 x 13 x 13

            nn.Conv2d(256, 384, kernel_size=3, padding=1),  # N x 384 x 13 x 13
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # N x 384 x 13 x 13
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # N x 256 x 13 x 13
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)  # N x 256 x 6 x 6
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)
        )

    #     self._initialize_weights()

    # def _initialize_weights(self):
    #     for layer in self.modules():
    #         if isinstance(layer, nn.Conv2d):
    #             nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
    #             if layer.bias is not None:
    #                 nn.init.constant_(layer.bias, 0)
    #         elif isinstance(layer, nn.Linear):
    #             nn.init.normal_(layer.weight, 0, 0.01)
    #             nn.init.constant_(layer.bias, 1)

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

Training

In [14]:
# Assuming train_loader and valid_loader are defined

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_CLASSES = 100  # Class labels

model100 = AlexNet(num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()

learning_rate, weight_decay, momentum = 0.01, 0.0005, 0.9
optimizer = torch.optim.SGD(model100.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

# Approach for learning rate decay
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

# Train the model
num_epochs = 20
losses, training_accuracies, validation_accuracies = train(
    model100, criterion, train_loader, valid_loader, optimizer, lr_scheduler, num_epochs
)

# Save the model weights
torch.save(model100.state_dict(), 'alexnet_weights_100_coutout.pth') # save the model weights
torch.save(model100, 'alexnet_model_100_cutout.pth') # save the entire model
print(f'\nCutout Model weights saved successfully!')

Training Loop with Cutout... 
------Validation Loop --------
Epoch: 1 	Loss: 4.5517 	Training Accuracy: 0.0099 	Validation Accuracy: 0.0174
------Validation Loop --------
Epoch: 2 	Loss: 4.1103 	Training Accuracy: 0.0402 	Validation Accuracy: 0.0720
------Validation Loop --------
Epoch: 3 	Loss: 3.7330 	Training Accuracy: 0.0896 	Validation Accuracy: 0.1166
------Validation Loop --------
Epoch: 4 	Loss: 3.4080 	Training Accuracy: 0.1422 	Validation Accuracy: 0.2104
------Validation Loop --------
Epoch: 5 	Loss: 3.1409 	Training Accuracy: 0.1892 	Validation Accuracy: 0.2432
------Validation Loop --------
Epoch: 6 	Loss: 2.9023 	Training Accuracy: 0.2344 	Validation Accuracy: 0.2956
------Validation Loop --------
Epoch: 7 	Loss: 2.6843 	Training Accuracy: 0.2778 	Validation Accuracy: 0.3340
------Validation Loop --------
Epoch: 8 	Loss: 2.4846 	Training Accuracy: 0.3107 	Validation Accuracy: 0.3858
------Validation Loop --------
Epoch: 9 	Loss: 2.3134 	Training Accuracy: 0.3464 	Validati

Plots

In [15]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a subplot with two rows and one column
fig = make_subplots(rows=2, cols=1, subplot_titles=("Accuracy", "Loss"))

# Add training accuracy to the first subplot
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=training_accuracies, mode='lines', name='Train Accuracy'), row=1, col=1)

# Add validation accuracy to the first subplot
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=validation_accuracies, mode='lines', name='Valid Accuracy'), row=1, col=1)

# Add training loss to the second subplot
fig.add_trace(go.Scatter(x=list(range(num_epochs)), y=losses, mode='lines', name='Loss'), row=2, col=1)

# Update the layout with the title
fig.update_layout(title='Model Accuracy and Loss - CutOut (CIFAR100)')

# Show the figure
fig.show()
# Save the imnage
plt.savefig('accuracy_plot_CutOut_cifar100.png')

<Figure size 640x480 with 0 Axes>

Model evaluation

In [17]:
# Evaluate the Model
model100 = AlexNet(num_classes=NUM_CLASSES).to(device)
model100.load_state_dict(torch.load('/kaggle/working/alexnet_weights_100_coutout.pth'))
model100.eval()

# Evaluate the model on the test set
top1_error, top5_error, test_accuracy = test(model100, test_loader, mode='Test')
print(f"Top-1 Error: {top1_error * 100:.2f}%,  Top-5 Error: {top5_error * 100:.2f}%,  Test Accuracy: {test_accuracy*100:.2f}%")

------Test Loop --------
Top-1 Error: 44.74%,  Top-5 Error: 17.35%,  Test Accuracy: 55.26%
